# Requirement:

In [ ]:
geojson == 3.1.0
pymongo == 4.7.2
pandas == 2.1.4
folium == 0.16.0
numpy == 1.26.4
numba == 0.59.0
geopy == 2.4.1
shapely == 2.0.4
geopandas == 0.14.4

In [2]:
import sys
sys.path.insert(0, './library/')
import zipfile
import os
import time
import pymongo as pym
import pandas as pd
import folium
import numpy as np
import requests
import numba

import math
import geopy
from shapely.geometry import Polygon, MultiPolygon, Point, mapping
from geopy.distance import geodesic,great_circle
from folium.plugins import FastMarkerCluster
from datetime import datetime
from geopy.distance import geodesic,great_circle
from pymongo import MongoClient

from pathlib import Path
import geopandas as gpd
import geojson
from IPython.core.display import display, HTML

from library import libStopsPoints, libConnections, libHex
 

C:\Users\mourahou\AppData\Local\Temp\ipykernel_18596\3436562659.py:25: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Data:

1. gtfs file of the city.
 ->[repository of gtfs file https://transitfeeds.com/]
2. pbf file of [openstreetmap](openstreetmap.org) extract from of the city/region of interest. ->[repository of osm extract: http://download.geofabrik.de/]

# Url and paths  [**set it!**]

In [ ]:
city = 'Paris'
scenario_name = 'basecase' # basecase without drt

# file paths and addresses to be provided:
# population data
shpPath = 'Paris/shp/pop.shp'
popCollectionName = "pop"
popField = "pop"

# Study area
# Study area defines the area in which the accessibility calaculation is executes
study_area_shp_path = 'Paris/shp/studyarea.shp'


directoryGTFS = ""+ city + '/gtfs/'+ scenario_name +'/' # directory of the gtfs files.

gtfs_prep_toggle = True
urlMongoDb = "mongodb://localhost:27017/"  # url of the mongodb database
urlMongoDbPop = "mongodb://localhost:27017/" # url of the mongodb database for population
urlServerOsrm = 'http://localhost:5000/' # url of the osrm server of the city

# required parameters
# the date must be in the interval of validity of the gtfs files, check it in the "calendar.txt" and "calendar_dates.txt" files inside the gtfs zip files.
# the day for the computation of accessibility quantitites
day = '20220307'
dayName = "monday" 

# List of starting time for computing the isochrones
# Sync to operation hours of drt, pt and conversion timeframe
timeList = list(range(7, 11, 1)) # -->[7,8,9,10,11]
# timeList = [7,10,13,16,19,22]
hStart = timeList[0]*3600 # converting to seconds

# parameters of walking distance
timeWalk = 15 * 60  # seconds
velocityWalk = 1.39  # m/s ***5km/h***
distanceS = timeWalk * velocityWalk

# Parameters that define the resolution and extention of tesselletion and the maximum of the walking time
# grid step of the hexagonal tesselletion in kilometers
gridEdge = 1

# Set check4stops = False if cells / hexagones should be included that do not have stops within.
# Set check4stops = False for preprocessing prior to dynamic mode to gtfs convertion
# Set check4stops = True for citychrone accessibility analysis
check4stops = False

# Start of the computation

### add population data

In [ ]:

shapefile = gpd.read_file(shpPath)
shapefile.to_file("Paris/shp/CentresOfInterest.geojson", driver='GeoJSON')
with open("Paris/shp/CentresOfInterest.geojson",encoding="utf8") as f:
    gj = geojson.load(f)
features = gj['features']
gtfsDB["POP"].drop()
gtfsDB["POP"].insert_many(features)


In [ ]:
# load Gtfs data to mongodb database
listOfFile = ['stops.txt', 'routes.txt', 'trips.txt', 'calendar.txt', 'calendar_dates.txt',
              'stop_times.txt']  
libStopsPoints.loadGtfsFile(gtfsDB, directoryGTFS, city, listOfFile)

## Fill the database with the connections

In [ ]:
libConnections.readConnections(gtfsDB, city, directoryGTFS, day, dayName)

## remove stops with no connections and add to each stop the pos field

In [ ]:
libStopsPoints.removingStopsNoConnections(gtfsDB, city)
libStopsPoints.setPosField(gtfsDB, city)

In [ ]:
libConnections.updateConnectionsStopName(gtfsDB, city)

# Tassel with exagons

### List of all stops

In [24]:
stopsList = libStopsPoints.returnStopsList(gtfsDB, city)

tot stop 35143  stop error : 0


## Compute the box that include all stops
The edge of such box are enlarged by distanceS.

In [25]:

display(HTML('<h1>All stops of the public transport present in the gtfs files</h1>'))
bbox = libStopsPoints.boundingBoxStops(stopsList)
libStopsPoints.mapStops(bbox, stopsList)

C:\Users\mourahou\AppData\Local\Temp\ipykernel_18596\1798799915.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Tassel the box with exagons.

In [ ]:
hexBin, pointBin = libHex.hexagonalGrid(bbox, gridEdge, gtfsDB['stops'], distanceS, city,check4stops)

In [27]:

libHex.insertPoints(pointBin, city, gtfsDB)


In [ ]:

display(HTML('<h1>First tesselletion of the area served by public transport</h1>'))
latlon = list(reversed(gtfsDB['points'].find_one({'city':city})['point']['coordinates']))
map_osm = folium.Map(location=latlon, zoom_start=9);
map_osm.choropleth(libHex.unionHexs(pointBin),  fill_color='#3288bd',fill_opacity=0.3, line_color='#3288bd',line_weight=2, line_opacity=1)
map_osm

## Find the hex with walkingTime less than timeWalk from a stops

In [28]:

libHex.pointsServed(gtfsDB, stopsList, urlServerOsrm, distanceS, timeWalk, city)

In [29]:
print("Number of hexagons: {0}".format(gtfsDB['points'].count_documents({'served':True, 'city':city})))

Number of hexagons: 11092


## Setting field "pos" for points for performance

In [30]:
from library import libHex 
libHex.settingHexsPos(gtfsDB, city)

In [ ]:
# from libHex import showHexs
from IPython.core.display import display, HTML
display(HTML('<h1>Tesselletion of the area served by the public transport</h1>'))
libHex.showHexs(gtfsDB, city, 10)

## Setting Population of Hexagons

In [ ]:
# from libHex import setHexsPop
import library
from library import libHex
if urlMongoDbPop != "" and popCollectionName != "":
    clientPop = pym.MongoClient(urlMongoDbPop)
    popDb = clientPop[popDbName]
    popCollection = popDb[popCollectionName]
    libHex.setHexsPop(gtfsDB, popCollection, popField, city)
else:
    print("Population NOT INSERTED!")

res = gtfsDB['points'].update_many({'pop':{'$exists':False}}, {'$set':{'pop':0}})
print("n° of matched hexagons with population Polygons: {0} \n \
not matched: {1} (setted to zero)".format(gtfsDB['points'].count_documents({'pop':{'$exists':True}}),
                                                                                     res.modified_count))

# Adding the walking time between stops and points

In [32]:
from library import libStopsPoints 
libStopsPoints.computeNeigh(gtfsDB, urlServerOsrm, distanceS, timeWalk,  city)

C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


# Compute quantities and observable

TimeList is the list of starting time for computing the isochrones

In [33]:
timeList = list(range(6,22,2))#[7,10,13,16,19,22] # List of starting time for computing the isochrones
#timeList = [7,10,13,16,19,22] # List of starting time for computing the isochrones
hStart = timeList[0]*3600

### List of connections

In [34]:
from library import libConnections 
arrayCC = libConnections.makeArrayConnections(gtfsDB, hStart, city)

start making connections array
done recover all cc 2989768
cenverted
Num of connection 2989768


### List of list of the points and stops neighbors

In [35]:
from library import libStopsPoints 
arraySP = libStopsPoints.listPointsStopsN(gtfsDB, city)

## Compute accessibility quantities

In [ ]:
import imp
import libAccessibility
import icsa


imp.reload(libAccessibility)
from icsa import computeAccessibilities
imp.reload(icsa)
listAccessibility = ['velocityScore','socialityScore','velocityScore1h', 'socialityScore1h']

computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    #gtfsDB['isochrones'].delete_many({'city':city})
    pass
for timeStart in timeList:
    timeStart *= 3600
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
    computeAccessibilities(city, timeStart, arrayCC, arraySP, gtfsDB, computeIsochrone, timeStart/3600 == timeList[0], listAccessibility=listAccessibility)

Time Isochrone Start: 6.0


C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#sessi

Time Isochrone Start: 8.0ore : 2.5, Sociality Score : 1.2, time to finish : 0.0h, 0.0 mmmmm


C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#sessi

Time Isochrone Start: 10.0re : 2.4, Sociality Score : 0.0, time to finish : 0.0h, 0.0 mmmmm


...

C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#sessi

Time Isochrone Start: 14.0re : 2.4, Sociality Score : 0.0, time to finish : 0.0h, 0.1 mmmmmm


C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#sessi

Time Isochrone Start: 16.0re : 2.4, Sociality Score : 0.0, time to finish : 0.0h, 0.0 mmmmm


C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#sessi

## Compute averages of the accessiblity quantities computed

In [ ]:
from libStopsPoints import computeAverage
computeAverage(listAccessibility, gtfsDB, city)

# RESULTS

## maps

In [ ]:
from libHex import reduceGeojsonInShellSubField
from IPython.core.display import display, HTML

field1 = 'velocityScore'
field2 = 'avg'
color = ['#993404', "#f16913", "#fdae6b", '#74c476', '#31a354', '#006d2c', "#6baed6", "#4292c6", "#2171b5", '#08519c', '#f768a1', '#dd3497', '#ae017e', '#49006a'];
shell = [0., 2., 4., 5, 6., 7, 8., 9, 10., 11, 12., 13, 15, 17.];
print ("number of hexs in total", gtfsDB['points'].find({field1:{'$exists':True}, 'city':city}).count())
res = reduceGeojsonInShellSubField(list(gtfsDB['points'].find({'city':city})), field1, field2, color, shell)
#res = showMapHexRedux(city, gtfsDB['points'], field = field, shell = shell, save=True)

display(HTML('<h1>Velocity Score</h1>'))

res[1]

In [ ]:
from libHex import reduceGeojsonInShellSubField
from IPython.core.display import display, HTML

field1 = 'socialityScore'
field2 = 'avg'
color = ["#000000","rgb(95, 95, 95)","rgb(180, 180, 180)","rgb(8, 48, 107)","rgb(15, 87, 159)","rgb(47, 126, 188)","rgb(109, 174, 213)","rgb(181, 212, 233)","rgb(253, 202, 148)",
"rgb(253, 176, 122)","rgb(250, 142, 93)","rgb(241, 108, 73)","rgb(224, 69, 48)","rgb(243, 105, 163)","rgb(224, 62, 152)","rgb(153, 3, 124)","rgb(73, 0, 106)"]
shell = [0, 50000, 100000, 200000, 300000, 400000, 500000, 600000,700000,800000, 900000, 1000000,1500000, 2000000,2500000, 3000000];
print ("number of hexs in total", gtfsDB['points'].find({field1:{'$exists':True}, 'city':city}).count())
res = reduceGeojsonInShellSubField(list(gtfsDB['points'].find({'city':city})), field1, field2, color, shell)
#res = showMapHexRedux(city, gtfsDB['points'], field = field, shell = shell, save=True)

display(HTML('<h1>Sociality Score</h1>'))

res[1]

# Saving File
Make ZIP file containig all the public transports information needed in order to add the city to the [citychrone](www.citychrone.org) platform.

In [ ]:
import saveData
newScenario=True # If True in the citychrone platform tensting new scenario on the city is allowed.
from saveData import makeZipCitychrone
if 'arrayCC' in locals():
    makeZipCitychrone(city, gtfsDB, arrayCC, newScenario=newScenario, urlServerOsrm=urlServerOsrm)
else:
    makeZipCitychrone(city, gtfsDB, newScenario=True)